<a href="https://colab.research.google.com/github/huggingface/deep-rl-class/blob/main/notebooks/unit3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit 3: Глубокое Q-Обучение с помощью игр Atari 👾 с использованием RL Baselines3 Zoo

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/thumbnail.jpg" alt="Блок 3 Миниатюра">

В этом блокноте **вы обучите агента Глубокого Q-Обучения**, играющего в Space Invaders, используя [RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo), обучающий фреймворк, основанный на [Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/), который предоставляет скрипты для обучения и оценки агентов, оптимизации гиперпараметров, построения графиков результатов и записи видео.

Мы используем интеграцию [RL-Baselines-3 Zoo, ванильную версию Deep Q-Learning](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) без расширений, таких как Double-DQN, Dueling-DQN и Prioritized Experience Replay.

⬇️ Вот пример того, чего **вы сможете достичь**. ⬇️

In [1]:
%%html
<video controls autoplay><source src="https://huggingface.co/ThomasSimonini/ppo-SpaceInvadersNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

### 🎮 Среды: 

- [SpacesInvadersNoFrameskip-v4](https://gymnasium.farama.org/environments/atari/space_invaders/)

Разницу между версиями Space Invaders можно увидеть здесь 👉 https://gymnasium.farama.org/environments/atari/space_invaders/#variants

### 📚 RL-библиотека: 

- [RL-Baselines3-Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)

## Задачи этого блокнота 🏆
В конце блокнота вы сможете:
- Сможете глубже понять **как работает RL Baselines3 Zoo**.
- Сможете **выложить своего обученного агента и код на Hub** с красивым видеоповтором и оценкой 🔥.




## Этот ноутбук взят из курса "Глубокое обучение с подкреплением".
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration"/>

В этом бесплатном курсе вы:

- 📖 Изучите глубокое обучение с подкреплением в **теории и на практике**.
- 🧑‍💻 Научитесь **использовать известные библиотеки Deep RL**, такие как Stable Baselines3, RL Baselines3 Zoo, CleanRL и Sample Factory 2.0.
- 🤖 Обучите **агентов в уникальных средах**.  

Чтобы узнать больше посмотрите 📚 учебный план 👉 https://simoninithomas.github.io/deep-rl-course

Не забудьте **<a href="http://eepurl.com/ic5ZUD">записаться на курс</a>** (мы собираем ваш адреса электронной почты, чтобы иметь возможность **высылать вам ссылки, когда каждый модуль будет опубликован, и предоставлять вам информацию о задачах и обновлениях).**


Лучший способ поддерживать связь - присоединиться к нашему discord-серверу, чтобы обмениваться информацией с сообществом и с нами 👉🏻 https://discord.gg/ydHrjt3WP5

## Предварительные условия 🏗️

Прежде чем погрузиться в ноутбук, необходимо:

🔲 📚 **[Изучить Глубокое Q-Обучение, прочитав Блок 3](https://huggingface.co/deep-rl-course/unit3/introduction)**  🤗 

Мы постоянно стараемся улучшить наши руководства, поэтому **если вы обнаружите какие-то проблемы в этом блокноте**, пожалуйста, [откройте issue на репозитории Github](https://github.com/huggingface/deep-rl-class/issues).

# Давайте обучим агента с Глубоким Q-Обучением игре в Space Invaders от Atari 👾 и загрузим его в Hub.

Мы настоятельно рекомендуем студентам **использовать Google Colab для проведения практических занятий, а не запускать их на своих персональных компьютерах**.

Использование Google Colab **позволяет сосредоточиться на обучении и экспериментах, не заботясь о технических аспектах настройки среды**.

Чтобы подтвердить практическое занятие для сертификации, необходимо загрузить обученную модель в Hub и **получить результат >= 200**.

Чтобы узнать свой результат, зайдите в таблицу лидеров и найдите свою модель, **result = mean_reward - std of reward**

Более подробную информацию о процессе сертификации можно найти в этом разделе 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Совет 💡

Лучше запускать этот блокнот в виде копии с Google Drive, чтобы **в случае тайм-аута** у вас оставался сохраненный блокнот на Google Drive и не нужно было заполнять все заново.

Для этого можно либо нажать `Ctrl + S`, либо `File > Save a copy in Google Drive.`.

Также мы собираемся **обучать модель в течение 90 минут с 1М временных шагов**. Введя `!nvidia-smi`, вы узнаете, какой GPU вы используете.

А если вы хотите обучить более 10 миллионов шагов, то это займет около 9 часов, что может привести к прерыванию работы Colab из за ограничения по времени. В этом случае я рекомендую запустить его на локальном компьютере (или в другом месте). Просто щелкните на: `File>Download`.

## Настройка GPU 💪.
- Чтобы **ускорить обучение агента, мы будем использовать GPU**. Для этого перейдите в меню `Runtime > Change Runtime type`.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step1.jpg" alt="GPU Шаг 1">

- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

# Установка RL-Baselines3 Zoo и его зависимостей 📚

Если вы видите сообщение `ERROR: pip's dependency resolver does not currently take account all the packages that are installed.` **это нормально и не является критической ошибкой**, это конфликт версий. Но нужные нам пакеты установлены.

In [ ]:
# На данный момент мы устанавливаем это обновление RL-Baselines3 Zoo
!pip install git+https://github.com/DLR-RM/rl-baselines3-zoo@update/hf

В ТОМ И ТОЛЬКО В ТОМ СЛУЧАЕ, ЕСЛИ ВЫШЕУКАЗАННАЯ ВЕРСИЯ БОЛЬШЕ НЕ СУЩЕСТВУЕТ. РАСКОММЕНТИРУЙТЕ И УСТАНОВИТЕ ПРИВЕДЕННУЮ НИЖЕ ВЕРСИЮ

In [ ]:
#!pip install rl_zoo3==2.0.0a9

In [ ]:
!apt-get install swig cmake ffmpeg

Чтобы иметь возможность использовать игры Atari в Gymnasium, нам необходимо установить пакет atari. И accept-rom-license для загрузки rom-файлов (файлов игр).

In [ ]:
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

## Создание виртуального дисплея 🔽.

В процессе работы с блокнотом нам потребуется создать видеозапись повтора. Для этого, с помощью Colab, **нам необходимо иметь виртуальный экран, чтобы иметь возможность визуализировать среду** (и, соответственно, записывать фреймы). 

Поэтому в следующей ячейке мы установим библиотеки, создадим и запустим виртуальный экран 🖥.

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Виртуальный дисплей
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Обучение нашего агента с Глубоким Q-Обучением игре в Space Invaders 👾

Чтобы обучить агента с помощью RL-Baselines3-Zoo, нам нужно сделать всего две вещи:

1. Создать файл конфигурации гиперпараметров, который будет содержать наши обучающие гиперпараметры, под названием `dqn.yml`.

Это пример шаблона:

```
SpaceInvadersNoFrameskip-v4:
  env_wrapper:
    - stable_baselines3.common.atari_wrappers.AtariWrapper
  frame_stack: 4
  policy: 'CnnPolicy'
  n_timesteps: !!float 1e7
  buffer_size: 100000
  learning_rate: !!float 1e-4
  batch_size: 32
  learning_starts: 100000
  target_update_interval: 1000
  train_freq: 4
  gradient_steps: 1
  exploration_fraction: 0.1
  exploration_final_eps: 0.01
  # If True, you need to deactivate handle_timeout_termination
  # in the replay_buffer_kwargs
  optimize_memory_usage: False
```

Здесь мы видим следующее:
- Мы используем `Atari Wrapper` для предварительной обработки входного сигнала (уменьшение кадра, задания режима grayscale, стэкирования 4 фреймов).
- Мы используем `CnnPolicy`, поскольку для обработки кадров мы используем сверточные слои
- Обучение проводится на 10 миллионах `n_timesteps`. 
- Размер памяти (Повтор опыта, Experience Replay) - 100000, т.е. количество шагов опыта, которое вы сохранили для повторного обучения агента.

💡 Мой совет - **сократить количество шагов обучения до 1M,** что займет около 90 минут на P100. `!nvidia-smi` подскажет, какой GPU вы используете. При 10 миллионов шагов это займет около 9 часов, что может привести к завершению работы Colab по timing out. Я рекомендую запустить эту программу на локальном компьютере (или в другом месте). Просто щелкните на: `File>Download`. 

С точки зрения оптимизации гиперпараметров, мой совет - сосредоточиться на этих 3 гиперпараметрах:
- `learning_rate`
- `buffer_size ("Размер памяти опыта")`.
- `batch_size`

В качестве хорошей практики необходимо **посмотреть документацию, чтобы понять, что делает каждый гиперпараметр**: https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html#parameters



2. Запускаем обучение и сохраняем модели в папке `logs` 📁.

- Определяем алгоритм после `--algo`, задаём путь сохранения модели после `-f` и определяем конфигурацию гиперпараметров после `-c`.

In [ ]:
!python -m rl_zoo3.train --algo ________ --env SpaceInvadersNoFrameskip-v4  -f _________  -c _________

#### Решение

In [ ]:
!python -m rl_zoo3.train --algo dqn  --env SpaceInvadersNoFrameskip-v4 -f logs/ -c dqn.yml

## Давайте оценим нашего агента 👀.
- RL-Baselines3-Zoo предоставляет `enjoy.py`, скрипт на языке python для оценки нашего агента. В большинстве библиотек RL мы называем сценарий оценки `enjoy.py`.
- Давайте оценим его на 5000 временных шагах 🔥.

In [ ]:
!python -m rl_zoo3.enjoy  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps _________  --folder logs/ 

#### Решение

In [ ]:
!python -m rl_zoo3.enjoy  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --no-render  --n-timesteps 5000  --folder logs/

## Публикация обученной модели на Hub 🚀.
Теперь, когда мы увидели, что получили хорошие результаты после обучения, мы можем опубликовать нашу обученную модель на Hub 🤗 с помощью одной строки кода.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/space-invaders-model.gif" alt="Модель Space Invaders">

Используя `rl_zoo3.push_to_hub` **вы оцениваете, записываете реплей, генерируете карточку модели вашего агента и отправляете ее в Hub**.

Таким образом:
- Вы можете **показать нашу работу** 🔥.
- Вы можете **визуализировать игру вашего агента** 👀.
- Вы можете **поделиться с сообществом агентом, который может быть использован другими** 💾
- Вы можете **зайти в таблицу результатов 🏆 и посмотреть, насколько хорошо ваш агент играет по сравнению с вашими одногруппниками** 👉  https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard

Чтобы получить возможность поделиться своей моделью с сообществом, необходимо выполнить еще три шага:

1️⃣ (Если это еще не было сделано) создайте учетную запись  HF ➡ https://huggingface.co/join

2️⃣ Выполните авторизацию, а затем сохраните свой токен аутентификации с сайта Hugging Face.
- Создайте новый токен (https://huggingface.co/settings/tokens) **с ролью write**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/create-token.jpg" alt="Создание токена HF">

- Скопируйте токен 
- Выполните ячейку ниже и вставьте токен

In [ ]:
from huggingface_hub import notebook_login # Войдите в нашу учетную запись Hugging Face, чтобы иметь возможность загружать модели в Hub.
notebook_login()
!git config --global credential.helper store

Если вы не хотите использовать Google Colab или Jupyter Notebook, то вместо этого необходимо использовать данную команду: `huggingface-cli login`

3️⃣ We're now ready to push our trained agent to the 🤗 Hub 🔥

Запустим файл push_to_hub.py для загрузки нашего обученного агента в Hub.

`--repo-name`: Имя репозитория

`-orga`: Ваше имя пользователя Hugging Face

`-f`: Папка, в которой находится обученная модель (в нашем случае `logs`)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit3/select-id.png" alt="Выбор Id">

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name _____________________ -orga _____________________ -f logs/

#### Решение

In [ ]:
!python -m rl_zoo3.push_to_hub  --algo dqn  --env SpaceInvadersNoFrameskip-v4  --repo-name dqn-SpaceInvadersNoFrameskip-v4  -orga ThomasSimonini  -f logs/

###.

Поздравляем 🥳 Вы только что обучили и загрузили свой первый агент с Глубоким Q-Обучением, использующий RL-Baselines-3 Zoo. В приведенном выше скрипте должна была появиться ссылка на репозиторий моделей, например https://huggingface.co/ThomasSimonini/dqn-SpaceInvadersNoFrameskip-v4. Перейдя по этой ссылке, вы сможете:

- Увидеть **видеопревью вашего агента** справа. 
- Щелкните " Files and versions", чтобы увидеть все файлы в репозитории.
- Щелкните "Use in stable-baselines3", чтобы получить фрагмент кода, показывающий, как загрузить модель.
- Карточка модели (файл `README.md`), в которой дается описание модели и используемых гиперпараметров.

Под капотом Hub использует репозитории на основе git (не волнуйтесь, если вы не знаете, что такое git), что означает, что вы можете обновлять модель новыми версиями по мере того, как вы экспериментируете и улучшаете своего агента.

**Сравнивайте результаты своих агентов с результатами своих одногруппников**, используя [таблицу результатов](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) 🏆

## Загрузка мощной обученной модели 🔥.
- Команда Stable-Baselines3 загрузила **более 150 обученных агентов глубокого обучения с подкреплением на Hub**.

Вы можете найти их здесь: 👉 https://huggingface.co/sb3

Некоторые примеры:
- "Asteroids": https://huggingface.co/sb3/dqn-AsteroidsNoFrameskip-v4
- Beam Rider: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4
- Breakout: https://huggingface.co/sb3/dqn-BreakoutNoFrameskip-v4
- Road Runner: https://huggingface.co/sb3/dqn-RoadRunnerNoFrameskip-v4

Загрузим агента, играющего в Beam Rider: https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4

In [ ]:
%%html
<video controls autoplay><source src="https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4/resolve/main/replay.mp4" type="video/mp4"></video>

1. Загружаем модель с помощью `rl_zoo3.load_from_hub` и помещаем ее в новую папку, которую можно назвать `rl_trained`

In [ ]:
# Загрузка модели и сохранение ее в папке logs/
!python -m rl_zoo3.load_from_hub --algo dqn --env BeamRiderNoFrameskip-v4 -orga sb3 -f rl_trained/

2. Оценим, модель на 5000 временных шагов

In [ ]:
!python -m rl_zoo3.enjoy --algo dqn --env BeamRiderNoFrameskip-v4 -n 5000  -f rl_trained/ --no-render

Почему бы не попробовать обучить своего собственного **Агента с Глубоким Q-Обучением, играя в BeamRiderNoFrameskip-v4? 🏆.**

Если хотите попробовать, посмотрите https://huggingface.co/sb3/dqn-BeamRiderNoFrameskip-v4#hyperparameters **в карточке модели есть гиперпараметры обученного агента.**

Однако поиск гиперпараметров может оказаться непростой задачей. К счастью, в следующем блоке мы увидим, как можно **использовать Optuna для оптимизации гиперпараметров 🔥.**


## Некоторые дополнительные задачи  🏆
Лучший способ научиться **это попробовать все самому**!

В таблице результатов [Leaderboard] (https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) вы найдете своих агентов. Сможете ли вы подняться на вершину?

Вот список сред, в которых вы можете попробовать обучить своего агента:
- BeamRiderNoFrameskip-v4
- BreakoutNoFrameskip-v4 
- EnduroNoFrameskip-v4
- PongNoFrameskip-v4

Также, **если вы хотите научиться самостоятельно реализовывать Глубоким Q-Обучением**, вам обязательно стоит посмотреть на реализацию CleanRL: https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/dqn_atari.py

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit4/atari-envs.gif" alt="Среды"/>

________________________________________________________________________
Поздравляю с окончанием этой части!

Если вы все еще чувствуете себя сбитым с толку всеми этими элементами... это абсолютно нормально! **Так было и со мной, и со всеми, кто изучал RL.**.

Потратьте время, чтобы действительно **усвоить материал, прежде чем продолжать и пробовать дополнительные задачи**. Важно освоить эти элементы и иметь прочный фундамент.

В следующем блоке **мы познакомимся с [Optuna](https://optuna.org/)**. Одной из наиболее важных задач в глубоком обучении с подкреплением является поиск хорошего набора обучающих гиперпараметров. И Optuna - это библиотека, которая помогает автоматизировать этот поиск.






### Это курс, созданный с вашим участием 👷🏿‍♀️.

Наконец, мы хотим итеративно улучшать и обновлять курс, учитывая ваши отзывы. Если у вас есть пожелания, пожалуйста, заполните эту форму 👉 https://forms.gle/3HgA7bEHwAmmLfwh9.

Мы постоянно стараемся улучшить наши учебные пособия, поэтому **если вы обнаружили какие-то проблемы в этом блокноте**, пожалуйста, [откройте issue в репозитории Github](https://github.com/huggingface/deep-rl-class/issues).

До встречи во втором бонусном блоке! 🔥

### Продолжайте учиться, оставайтесь потрясающими 🤗